In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.neural_network import MLPClassifier

df = pd.read_csv("/Users/omanand/Auralis/data/processed/cleaned_data.csv")

X = df.drop(columns=["stability_label"])
y = df["stability_label"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42, stratify=y)


mlp = MLPClassifier(
    hidden_layer_sizes=(64,32),
    activation="relu",
    solver="adam", 
    max_iter=500,
    random_state=42
    )

mlp.fit(X_train,y_train)

y_pred = mlp.predict(X_test)
y_prob = mlp.predict_proba(X_test)[:, 1]


print("Neural-style Model Performance")
print(classification_report(y_test, y_pred))
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred)}\n")


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(mlp, X, y, cv=cv, scoring="roc_auc")
print("CV ROC-AUC scores:", cv_scores)
print("CV Mean ROC-AUC:", cv_scores.mean())

# Save model
joblib.dump(mlp, "/Users/omanand/Auralis/models/neural_instability_predictor.pkl")